# Análisis de sentimiento en noticias financieras y conferencias de resultados

### Integrantes

Pablo González Barón

### Objetivos

1. Clasificar las noticias financieras y las transcripciones de las conferencias de resultados de cada empresa como positivas, negativas o neutras para tomar decisiones de inversión sobre las acciones de cada empresa.
2. Analizar una posible correlación entre los retornos diarios de las acciones de cada empresa junto con el sentimiento (positivo, negativo o neutro) de las noticias financieras y las transcripciones de conferencias de resultados de cada día.
3. Caracterizar las noticias que causaron movimientos más agresivos en el precio de las acciones de ciertas empresas a las que estén relacionadas dichas noticias.

### Descripción

Extraer una base de datos que contiene información de noticias y titulares financieros pertenecientes a las acciones de las empresas que componen el índice financiero S&P500 para luego posteriormente clasificar las noticias de cada empresa con el fin de tomar decisiones de inversión y mediante un análisis de sentimiento observar la relación que existe con los retornos diarios en la bolsa de valores de dichas empresas, para identificar como la polaridad del sentimiento está relacionada a cada ganancia o pérdida diaria del mercado. Adicional a esto, se extrae una base de datos por medio de web scraping de transcripciones de conferencias de resultados, donde se aplica análisis de sentimiento utilizando dos modelos distintos y viendo la relación que puede tener con los retornos diarios en la bolsa de valores de las respectivas empresas.

### Datos disponibles

En la actualidad existen muchas bases de datos en Kaggle con noticias y titulares financieros de varias empresas, pero estas bases de datos no están actualizadas y las noticias/titulares no corresponden a la actualidad, por tanto en nuestro caso utilizaremos una API llamada Finnhub (https://finnhub.io/) que nos permite traer noticias y titulares financieros de cualquier empresa y elegir las fechas de las cuales queremos noticias, esto es importante para poder comparar contra los retornos diarios de cada empresa en esas fechas elegidas.

La versión gratuita de la API nos permite traer noticias de hasta un año atrás, esto es suficiente para poder realizar el análisis que necesitamos y entrenar el modelo a usar. La recolección de datos nos entrega la siguiente base de datos:

Para las transcripciones de las conferencias de resultados, se aplicó web scraping en la página web https://capedge.com utilizando la librería Selenium.

## Importar librerías

In [ ]:
import os
import nltk
import spacy
import pickle
import swifter
import warnings
import itertools
import matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from cucco import Cucco
from wordcloud import WordCloud, STOPWORDS
from spacytextblob.spacytextblob import SpacyTextBlob

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Importar datos
Las noticias financieras fueron extraídas en tres fechas distintas para agrupar la mayor cantidad de noticias en fechas distintas, por eso los datos vienen separados en 3 archivos distintos.

In [ ]:
d1 = pd.read_parquet('news_data.parquet.gzip')
d2 = pd.read_parquet('news_data_2.parquet.gzip')
d3 = pd.read_parquet('news_data_3.parquet.gzip')
data = pd.concat([d1, d2, d3], ignore_index=True)
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data[data.datetime < 0]

In [ ]:
# Removemos las fechas que son menores a 0, lo cual no tiene sentido.
data = data[data.datetime > 0].copy()
# Convertimos de formato unix a formato estándar.
data['datetime'] = pd.to_datetime(data['datetime'],unit='s')

## Análisis exploratorio

Graficamos un conteo de las noticias por categoria y por fuente

In [ ]:
sns.countplot(x=data.category);

In [ ]:
sns.countplot(y=data.source);

In [ ]:
data["headline"].groupby([data["datetime"].dt.year, data["datetime"].dt.month]).count().plot(kind="bar")

## Preprocesamiento del texto
En primera instancia aplicamos normalización con ayuda de la librería Cucco, pues esta librería nos permite elegir un conjunto específico de normalizaciones. En este caso no aplicamos lematizacion, pues al hacer esto podemos afectar el contexto del corpus al aplicar el pipeline de spacytextblob.

In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [ ]:
cucco = Cucco()

normalizations = [
    'remove_extra_white_spaces',
    ('replace_punctuation', {'replacement': ''}),
    ('replace_emojis', {'replacement': ''}),
    ('replace_symbols', {'replacement': ''}),
    ('replace_urls', {'replacement': ''}),
]

data['headline_clean'] = data['headline'].swifter.apply(lambda x : cucco.normalize(x, normalizations))
data['summary_clean'] = data['summary'].swifter.apply(lambda x : cucco.normalize(x, normalizations))

### spacytextblob

In [ ]:
# Aplicamos SpaCy NLP
data['headline_doc'] = data['headline_clean'].swifter.apply(lambda x : nlp(x))
data['summary_doc'] = data['summary_clean'].swifter.apply(lambda x : nlp(x))

In [ ]:
# Extraemos la polaridad de spacytextblob
data['headline_polarity'] = data['headline_doc'].swifter.apply(lambda x : x._.blob.polarity)
data['summary_polarity'] = data['summary_doc'].swifter.apply(lambda x : x._.blob.polarity)

In [ ]:
# Graficamos la distribución de la polaridad de los titulares y los resumenes de las noticias.
plt.hist(data.groupby(['related'])['headline_polarity'].mean(), alpha=.8, label='headline polarity')
plt.hist(data.groupby(['related'])['summary_polarity'].mean(), alpha=.8, label='summary polarity')
plt.grid()
plt.title('Histogram of summary column polarity scores across tickers')
plt.legend();

### FinBERT

In [ ]:
# Utilizamos el modelo FinBERT pre-entrenado de huggingface.
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
# Instanciamos el pipeline de analisis de sentimiento.
sentimentanalysis_nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
# Aplicamos el pipeline de analisis de sentimiento de forma paralelizada (usando todos los hilos disponibles del procesador) usando swifter.
df['headline_sentiment'] = df['headline'].swifter.apply(lambda x : sentimentanalysis_nlp(x)[0]['label'])

In [ ]:
# Guardamos los resultados parciales hasta ahora en un archivo pickle.
data.to_pickle("data.pkl")

### Evaluación de modelos y caracterización de noticias influyentes

In [ ]:
# Cargamos el archivo pickle guardado anteriormente.
df = pd.read_pickle('data.pkl')
# Para remover con el fin de caracterizar las noticias mas adelante
removal = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']

### Extraemos datos de los precios para todos los tickers usando yfinance

In [ ]:
import yfinance as yf

master_prices = {}
tickers = df.related.unique()

for ticker in tickers:
    print(ticker)
    t = yf.Ticker(ticker)
    
    # Extraemos unicamente los precios en las fechas para las cuales hay noticias.
    related_df = df[df.related == ticker].copy()
    price_history = t.history(start=related_df.datetime.min().strftime('%Y-%m-%d'), 
                              end=related_df.datetime.max().strftime('%Y-%m-%d'))[['Close']].reset_index()
    price_history['Ticker'] = ticker
    price_history.columns = ['datetime', 'close', 'ticker']
    master_prices[ticker] = price_history

### Calculamos las correlaciones de la polaridad capturada junto con el retorno, para cada ticker

In [ ]:
master_corrs = {}

for ticker in tickers:
    returns = master_prices[ticker].set_index('datetime')[['close']].pct_change().iloc[1:].reset_index()
    news = df[df.related == ticker]
    
    # Formateamos la columna 'datetime' ya que por esa columna haremos un inner join.
    news['datetime'] = news['datetime'].apply(lambda x : pd.to_datetime(x.strftime('%Y-%m-%d')))
    returns['datetime'] = returns['datetime'].apply(lambda x : pd.to_datetime(x.strftime('%Y-%m-%d')))
    polarities = news.groupby(['datetime'])[['headline_polarity']].mean().reset_index()
    res = pd.merge(polarities, returns, on='datetime', how='inner')
    
    # La correlación es calculada con el retorno del dia siguiente, asumiendo que el impacto de la noticia es un dia despues.
    res['headline_polarity'] = res['headline_polarity'].shift(1)
    res = res.iloc[1:]
    res = res.set_index('datetime').corr().iloc[0].iloc[1]
    master_corrs[ticker] = res
    
corrs = pd.DataFrame([master_corrs]).T.dropna()
corrs.columns = ['corr']

In [ ]:
font = {'size'   : 22}
matplotlib.rc('font', **font)

# Graficamos la distribución de las correlaciones
corrs.plot(kind='hist', color='#35B276')
plt.grid();
plt.savefig('spacytextblob_corr.png', bbox_inches='tight')

In [ ]:
# 5 tickers con mas correlación
corrs.sort_values(by='corr', ascending=False).head(5)

In [ ]:
# 5 tickers con menos correlación
corrs.sort_values(by='corr', ascending=False).tail(5)

### Correlaciones FinBERT

In [ ]:
# Función auxiliar para categorizar el sentimiento de FinBERT en 0, 1 o -1.
def categorize_sentiment(x):
    if x == 'neutral':
        return 0
    elif x == 'positive':
        return 1
    elif x == 'negative':
        return -1
    
# Función auxiliar para categorizar los valores flotantes de los retornos.
def categorize_returns(x, mean):
    if abs(x) < 1.5*abs(mean):
        return 0
    elif x > 0:
        return 1
    elif x < 0:
        return -1

### Calculamos las correlaciones del sentimiento capturado junto con el retorno, para cada ticker

In [ ]:
master_corrs_finbert = {}

for ticker in tickers:
    returns = master_prices[ticker].set_index('datetime')[['close']].pct_change().iloc[1:].reset_index()
    news = df[df.related == ticker]
    news['datetime'] = news['datetime'].apply(lambda x : pd.to_datetime(x.strftime('%Y-%m-%d')))
    returns['datetime'] = returns['datetime'].apply(lambda x : pd.to_datetime(x.strftime('%Y-%m-%d')))
    
    # Aplicamos las funciones auxiliares
    news['headline_sentiment_num'] = news['headline_sentiment'].apply(lambda x : categorize_sentiment(x))
    returns['close_num'] = returns['close'].apply(lambda x : categorize_returns(x, returns.close.mean()))
    sentiments = news.groupby(['datetime'])[['headline_sentiment_num']].agg(lambda x: 1 if pd.Series.mean(x) > 0 else -1).reset_index()
    res = pd.merge(sentiments, returns, on='datetime', how='inner')[['datetime', 'headline_sentiment_num', 'close_num']]
    
    # La correlación es calculada con el retorno del dia siguiente, asumiendo que el impacto de la noticia es un dia despues.
    res['headline_sentiment_num'] = res['headline_sentiment_num'].shift(1)
    res = res.iloc[1:]
    res = res.set_index('datetime').corr().iloc[0].iloc[1]
    master_corrs_finbert[ticker] = res
    
corrs_finbert = pd.DataFrame([master_corrs_finbert]).T.dropna()
corrs_finbert.columns = ['corr']

In [ ]:
font = {'size'   : 22}
import matplotlib
matplotlib.rc('font', **font)

# Graficamos la distribución de las correlaciones
corrs_finbert.plot(kind='hist', color='#35B276')
plt.grid();
plt.savefig('finbert_corr.png', bbox_inches='tight')

In [ ]:
# 5 tickers con mas correlación
corrs_finbert.sort_values(by='corr', ascending=False).head(5)

In [ ]:
# 5 tickers con menos correlación
corrs_finbert.sort_values(by='corr', ascending=False).tail(5)

### Caracterizar noticias que más movieron el mercado

In [ ]:
df = df.reset_index()

In [ ]:
master_movernews = []

for ticker in tickers:
    returns = master_prices[ticker].set_index('datetime')[['close']].pct_change().iloc[1:]
    
    # Extraemos los retornos atípicos
    lowerq = returns.quantile(.05).iloc[0]
    upperq = returns.quantile(.95).iloc[0]
    outliers = returns[(returns.close < lowerq) | (returns.close > upperq)].reset_index()
    
    # Formatemos la columna 'datetime'
    outliers['datetime'] = outliers['datetime'].apply(lambda x : pd.to_datetime(x.strftime('%Y-%m-%d')))
    ticker_news = df[df.related == ticker].copy()
    ticker_news['datetime'] = ticker_news['datetime'].apply(lambda x : pd.to_datetime(x.strftime('%Y-%m-%d')))
    
    news_outliers = {}
    for index, row in outliers.iterrows():
        
        # Utilizamos la fecha menos un día, asumiendo que el impacto de la noticia fue un día después.
        datestr = (row['datetime'] - timedelta(days=1)).strftime('%Y-%m-%d')
        
        res = ticker_news[ticker_news.datetime == datestr]['index'].tolist()
        
        if len(res) > 0:
            res = ', '.join([str(a) for a in res])
        else:
            res = '-'
            
        if row['close'] > 0:
            sentiment = 'positive'
        elif row['close'] < 0:
            sentiment = 'negative'
            
        news_outliers[datestr] = (res, sentiment)
      
    mover_news = pd.DataFrame(news_outliers).T.reset_index()
    if len(mover_news) > 0:
        mover_news.columns = ['datetime', 'id_news', 'sentiment']
        mover_news['ticker'] = ticker
        master_movernews.append(mover_news)

In [ ]:
master_movernews = pd.concat(master_movernews)
master_movernews['n_ofnews'] = master_movernews['id_news'].swifter.apply(lambda x : len(x.split(', ')) if x != '-' else 0)

### Extraer IDs de las noticias positivas y negativas

In [ ]:
negative_news = master_movernews[master_movernews.sentiment == 'negative'].copy()
negative_ids = negative_news.id_news.tolist()
negative_ids = [elem.split(', ') for elem in negative_ids if elem != '-']

# Convertimos una lista de listas en una sola lista con la libreria itertools.
negative_ids = list(itertools.chain.from_iterable(negative_ids))
negative_ids = [int(i) for i in negative_ids]

positive_news = master_movernews[master_movernews.sentiment == 'positive'].copy()
positive_ids = positive_news.id_news.tolist()
positive_ids = [elem.split(', ') for elem in positive_ids if elem != '-']

# Convertimos una lista de listas en una sola lista con la libreria itertools.
positive_ids = list(itertools.chain.from_iterable(positive_ids))
positive_ids = [int(i) for i in positive_ids]

In [ ]:
# Extraemos los textos crudos de las noticias positivas y negativas, teniendo en cuenta los IDs que extraimos anteriormente.
negative_texts = df[df['index'].isin(negative_ids)].copy()
positive_texts = df[df['index'].isin(positive_ids)].copy()

In [ ]:
negative_texts.shape, positive_texts.shape

### Lematizamos el texto del headline y eliminamos stopwords para aplicar la caracterización (nube de palabras y TM)

In [ ]:
negative_texts['headline_tm'] = negative_texts['headline_doc'].swifter.apply(lambda x : [token.lemma_.lower() for token in x if token.pos_ not in removal and not token.is_stop and token.is_alpha])
positive_texts['headline_tm'] = positive_texts['headline_doc'].swifter.apply(lambda x : [token.lemma_.lower() for token in x if token.pos_ not in removal and not token.is_stop and token.is_alpha])

In [ ]:
positive_texts.shape, negative_texts.shape

In [ ]:
# Convertimos una lista de listas en una sola lista con la libreria itertools.
negwords = list(itertools.chain.from_iterable(negative_texts.headline_tm.tolist()))
poswords = list(itertools.chain.from_iterable(positive_texts.headline_tm.tolist()))

In [ ]:
len(negwords), len(poswords)

### Creamos las nubes de palabras

In [ ]:
stopwords = STOPWORDS
# Aceptamos como máximo 200 palabras y aplicamos la nube de palabras sobre el corpus completo de palabras negativas o positivas.
wordcloud_neg = WordCloud(width=1920, height=1080, stopwords=stopwords, background_color="white", max_words=200).generate(' '.join(negwords))
wordcloud_pos = WordCloud(width=1920, height=1080, stopwords=stopwords, background_color="white", max_words=200).generate(' '.join(poswords))

In [ ]:
plt.rcParams['figure.figsize'] = (26, 6)

# Graficamos la nube de palabras de noticias negativas
plt.imshow(wordcloud_neg)
plt.savefig('negative_wordcloud.png', bbox_inches='tight')

In [ ]:
# Graficamos la nube de palabras de noticias positivas
plt.imshow(wordcloud_pos)
plt.savefig('positive_wordcloud.png', bbox_inches='tight')

### Topic Modeling (no se usó)

In [ ]:
# content = negative_texts['headline_tm'].tolist()
# id2word = Dictionary(content)
# id2word.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
# corpus = [id2word.doc2bow(text) for text in content]

In [ ]:
# topics = []
# score = []
# for i in range(1,10,1):
#     lda_model = LdaMulticore(corpus=corpus, id2word=id2word, iterations=10, num_topics=i, workers = 12, passes=10, random_state=100)
#     cm = CoherenceModel(model=lda_model, texts = content, corpus=corpus, dictionary=id2word, coherence='c_v')
#     topics.append(i)
#     score.append(cm.get_coherence())
# _=plt.plot(topics, score)
# _=plt.xlabel('Number of Topics')
# _=plt.ylabel('Coherence Score')
# plt.show()

In [ ]:
# lda_model = LdaMulticore(corpus=corpus, id2word=id2word, iterations=10, num_topics=3, workers=12, passes=10, random_state=100)

In [ ]:
# pyLDAvis.enable_notebook()
# p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# p

In [ ]:
# content = positive_texts['headline_tm'].tolist()
# id2word = Dictionary(content)
# id2word.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
# corpus = [id2word.doc2bow(text) for text in content]

In [ ]:
# lda_model = LdaMulticore(corpus=corpus, id2word=id2word, iterations=10, num_topics=3, workers=12, passes=10, random_state=100)

In [ ]:
# pyLDAvis.enable_notebook()
# p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# p

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c99f8b9f-730f-4242-befc-1feb4682d851' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>